In [23]:
import requests
import base64

In [24]:
CLIENT_ID = '648ece1061bb46c0bc2c7e360700847c'
CLIENT_SECRET = '3a6447a30cee4d869cc32393ce0cfabb'

In [25]:
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"

In [26]:
client_credentials_base64 = base64.b64encode(client_credentials.encode())

In [27]:
#requesting access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization':f'Basic {client_credentials_base64.decode()}'
}

In [28]:
data = {
    'grant_type':'client_credentials'
}

In [29]:
response = requests.post(token_url, data = data, headers = headers)

In [30]:
if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained succcessfully.")
else:
    print("Error obtaining accesstoken.")
    exit()

Access token obtained succcessfully.


In [31]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [34]:
def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [38]:
playlist_id = '37i9dQZF1DX76Wlfdnj7AP'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
print(music_df)

                     Track Name                      Artists  \
0                   Lovin On Me                  Jack Harlow   
1                        redrum                    21 Savage   
2                         Prada  cassö, RAYE, D-Block Europe   
3                      fukumean                        Gunna   
4               I'm Good (Blue)     David Guetta, Bebe Rexha   
..                          ...                          ...   
95                         BOTH       Tiësto, 21 Savage, BIA   
96            MONEY ON THE DASH          Elley Duhé, Whethan   
97  Another Love - Tiësto Remix            Tom Odell, Tiësto   
98           Paint The Town Red                     Doja Cat   
99                   In Ha Mood                    Ice Spice   

                     Album Name                Album ID  \
0                   Lovin On Me  6VCO0fDBGbRW8mCEvV95af   
1                american dream  2RRYaYHY7fIIdvFlvgb5vq   
2                         Prada  5MU0RmBSpoSxOPYBfcobD

In [41]:
# Building Music Reccomendation using Python
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

#renaming music dataframe to data
data = music_df

In [42]:
#function to calculate the weighted popularity scores based on release date
def caluclated_weight_popularity(release_date):
    
    #converting the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')
    
    #Calculating the timespan between release date and today's date
    time_span = datetime.now() - release_date
    
    #calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1/(time_span.days + 1)
    return weight

In [43]:
#Normaizing the music features using Min-Max scaling
scaler = MinMaxScaler()

music_features = data[['Danceability', 'Energy', 'Key', 
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values

music_features_scaled = scaler.fit_transform(music_features)

In [55]:
#CREATING MUSIC reccomendations based on music audio features
def content_based_reccomendations(input_song_name, num_reccomendations = 5 ):
    
    if input_song_name not in data['Track Name'].values:
        print(f" '{input_song_name}' not found in dataset. Please enter valud song name.")
        return
    
    #Getting the index of the input song in the music DataFrame
    input_song_index = data[data['Track Name']== input_song_name].index[0]
    
    #Caclulating similiarity scored based on music features, using cosine similiarity
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)
    
    #Getting the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_reccomendations + 1]
    
    #Getting the names of the most similar based based on content-based filtering
    content_based_reccomendations = data.iloc[similar_song_indices][['Track Name', 'Artists','Album Name',
                                                                    'Release Date', 'Popularity']]
    return content_based_reccomendations

In [59]:
#Getting Reccomendations for 'Paint The Town Red' by Doja Cat.

content_based_reccomendations('Paint The Town Red', 25)

,Track Name,Artists,Album Name,Release Date,Popularity
38,Ferrari,"James Hype, Miggy Dela Rosa",Ferrari,2022-04-01,84
87,MONEY,LISA,LALISA,2021-09-10,79
68,Lemonade (feat. NAV),"Internet Money, Gunna, Don Toliver, NAV",B4 The Storm,2020-08-28,81
11,Me Porto Bonito,"Bad Bunny, Chencho Corleone",Un Verano Sin Ti,2022-05-06,89
30,Freestyle,Lil Baby,Too Hard,2017-12-01,85
78,Vegas (From the Original Motion Picture Soundt...,Doja Cat,Vegas (From the Original Motion Picture Soundt...,2022-05-06,80
59,Private Landing (feat. Justin Bieber & Future),"Don Toliver, Justin Bieber, Future",Love Sick,2023-02-24,81
24,Give It To Me - Full Vocal Mix,Matt Sassari,Give It To Me (Full Vocal Mix),2021-10-22,86
64,INDUSTRY BABY (feat. Jack Harlow),"Lil Nas X, Jack Harlow",INDUSTRY BABY (feat. Jack Harlow),2021-07-23,81
97,Another Love - Tiësto Remix,"Tom Odell, Tiësto",Another Love (Tiësto Remix),2022-05-13,77
